In [ ]:
import pandas as pd
import numpy as np
import re
import string
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer

In [ ]:
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.ExcelFile("dataset_merged.xlsx").parse()

In [ ]:
df.head(2)

,created_at,id,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,geo,lang,user_id_str,conversation_id,conversation_id_str,media_url_https,media_type,username,Unnamed: 16,Unnamed: 17
0,Mon May 15 11:22:33 +0000 2023,1658070357826490112,1658070357826490112,@detikcom Saat kota yang lain sibuk berlomba a...,0,0,0,1,NaN,in,1124966023097080064,1658042035645929984,1658042035645929984,NaN,NaN,jalan_yuuk,NaN,NaN
1,Mon May 15 11:43:48 +0000 2023,1658075703882170112,1658075703882170112,"@think_tank_id @Dandhy_Laksono lol tolol, lu ...",0,0,0,0,NaN,in,133157248,1657544329018380032,1657544329018380032,NaN,NaN,zeronol0,NaN,NaN


In [ ]:
df = df[['created_at', 'id', 'full_text', 'reply_count', 'retweet_count', 'favorite_count', 'username', ]]

In [ ]:
df

,created_at,id,full_text,reply_count,retweet_count,favorite_count,username
0,Mon May 15 11:22:33 +0000 2023,1658070357826490112,@detikcom Saat kota yang lain sibuk berlomba a...,0,0,1,jalan_yuuk
1,Mon May 15 11:43:48 +0000 2023,1658075703882170112,"@think_tank_id @Dandhy_Laksono lol tolol, lu ...",0,0,0,zeronol0
2,Mon May 15 11:07:11 +0000 2023,1658066490258769920,"@jokowi @jokowi di merauke minyak tanah susah,...",0,0,0,PA_5060_AS
3,Mon May 15 11:00:55 +0000 2023,1658064915532480000,"Polres PPU Polda Kaltim - Wujud Kepedulian, Po...",0,0,1,bagrenresppu
4,Mon May 15 11:15:36 +0000 2023,1658068608453410048,"Traffic Light Rusak, Jalan Borong Raya-Batua R...",0,0,0,raksulonline
...,...,...,...,...,...,...,...
3866,Sun Nov 27 06:16:00 +0000 2022,1596749661821280000,yaelah wargaðŸ˜¢ðŸ˜¢ https://t.co/2421r3esOn,131,255,1012,txtdrjkt
3867,Sun Nov 27 10:52:33 +0000 2022,1596819259673299968,"@KompasTV Ternyata, cuma menyisakan sampah dim...",1,0,2,SupratmanAndang
3868,Mon Nov 28 03:23:42 +0000 2022,1597068690335620096,@Dara_Cega Kasihan masyarakat dibodohi dibayar...,0,0,0,BKoentjahya
3869,Mon Nov 28 01:58:09 +0000 2022,1597047161245819904,@UdinBagol81 @matjan51670210 @Rasambahyar1 @se...,0,0,0,freensapalocha


In [ ]:
def cleanText(text):
  #hapus username twitter
  text=re.sub('@[^\s]+', ' ', str(text))
  #hapus https dan http
  text = re.sub(r"(?:\@|http?\://|https?\://|www)S+", "", str(text))
  #hapus tanda baca
  text = text.translate(str.maketrans(' ', ' ',string.punctuation))
  #karakter html dengan tanda petik
  text=re.sub('<.*?>', ' ', str(text))
  text = re.sub('[^a-zA-Z]', ' ', str(text))
  text = re.sub("\n", " ", str(text))
  text = re.sub(r"\b[a-zA-Z]\b", " ", str(text))
  text = ' '.join(text.split())
  return text

In [ ]:
def lowercase(text):
  text = text.lower()
  return text

In [ ]:
def filteringText(text):
  listStopwords = set(stopwords.words('indonesian'))
  filtered = []
  for txt in text:
    if txt not in listStopwords:
      filtered.append(txt)
  text = filtered
  return text


In [ ]:
df['clean_text'] = df['full_text'].apply(cleanText)

In [ ]:
df['clean_text'] = df['clean_text'].apply(lowercase)

In [ ]:
df.drop(['full_text'], axis = 1, inplace = True)

In [ ]:
regexp = RegexpTokenizer(r'\w+|$[0-9]+|\S+')
df['preprocessed_text'] = df['clean_text'].apply(regexp.tokenize)

In [ ]:
df['preprocessed_text'] = df['preprocessed_text'].apply(filteringText)

In [ ]:
df.drop_duplicates(subset = 'clean_text', inplace = True)

In [ ]:
save_df = df.to_csv("dataset_merged_clean.csv", index=False, header=True, index_label=None)

In [ ]:
df.head(5)

,created_at,id,reply_count,retweet_count,favorite_count,username,clean_text,preprocessed_text
0,Mon May 15 11:22:33 +0000 2023,1658070357826490112,0,0,1,jalan_yuuk,saat kota yang lain sibuk berlomba agar wisata...,"[kota, sibuk, berlomba, wisatawan, kotanya, im..."
1,Mon May 15 11:43:48 +0000 2023,1658075703882170112,0,0,0,zeronol0,tankid laksono lol tolol lu pikir keren gitu ...,"[tankid, laksono, lol, tolol, lu, pikir, keren..."
2,Mon May 15 11:07:11 +0000 2023,1658066490258769920,0,0,0,PA_5060_AS,di merauke minyak tanah susah selain jalan jug...,"[merauke, minyak, tanah, susah, jalan, rusak, ..."
3,Mon May 15 11:00:55 +0000 2023,1658064915532480000,0,0,1,bagrenresppu,polres ppu polda kaltim wujud kepedulian pols...,"[polres, ppu, polda, kaltim, wujud, kepedulian..."
4,Mon May 15 11:15:36 +0000 2023,1658068608453410048,0,0,0,raksulonline,traffic light rusak jalan borong rayabatua ray...,"[traffic, light, rusak, jalan, borong, rayabat..."


In [ ]:
lexicon = pd.read_csv('positive.tsv', sep='\t')

In [ ]:
lexicon_positive = dict()
with open('positive.tsv', 'r') as file:
  reader = csv.reader(file, delimiter='\t')
  for row in reader:
    lexicon_positive[row[0]] = row[1]

In [ ]:
lexicon_negative = dict()
with open('negative.tsv', 'r') as file:
  reader = csv.reader(file, delimiter='\t')
  for row in reader:
    lexicon_negative[row[0]] = row[1]

In [ ]:
def sentiment_analysis(text):
  score = 0
  for word in text:
    if (word in lexicon_positive):
      score = score + int(lexicon_positive[word])
  for word in text:
    if (word in lexicon_negative):
      score = score + int(lexicon_negative[word])
  polarity = ''
  if (score > 0):
    polarity = 'positive'
  elif (score < 0):
    polarity = 'negative'
  else:
    polarity = 'neutral'
  return score, polarity

In [ ]:
first_key = next(iter(lexicon_positive))
del lexicon_positive[first_key]

In [ ]:
first_keyy = next(iter(lexicon_negative))
del lexicon_negative[first_keyy]

In [ ]:
results = df['preprocessed_text'].apply(sentiment_analysis)
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]
print(df['polarity'].value_counts())

negative    2277
positive    1077
neutral      389
Name: polarity, dtype: int64


In [ ]:
df = df.drop(df.index[len(df)-1])

In [ ]:
df.to_excel('data_polarity.xlsx', index=False, header=True)

In [ ]:
df.isna().sum()

created_at           1
id                   1
reply_count          1
retweet_count        1
favorite_count       1
username             5
clean_text           0
preprocessed_text    0
polarity_score       0
polarity             0
dtype: int64